<a href="https://colab.research.google.com/github/KyoYoung-Lim/KyoYoung-s-Repository/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC%EC%99%80_%EC%8B%9C%EA%B0%81%ED%99%94_with_%EC%83%9D%EC%84%B1%ED%98%95AI_pda2_1_Data_cleaning___%ED%83%9D%EC%8B%9C%EC%9A%94%EA%B8%88_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%8B%A4%EB%A3%A8%EA%B8%B0_%5B%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%5D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 전처리와 시각화 with 생성형AI_pda2
2. Data cleaning -택시요금 데이터 다루기 [프로젝트]

Day 1 Project: 택시 이용내역 데이터_데이터를 정제

*   결측치 판단 및 처리
(   [힌트] 결측치의 유무와 비율을 살펴보고 적절한 방법으로 처리합니다.)
*   이상치 판단 및 처리
(   [힌트] 숫자형 변수 각각에 대해, 데이터의 index와 변수 값 사이의 scatter plot을 그려보고 이상치가 관찰될 경우 제거합니다.)




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
df_taxi = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataSet/trip.csv')
df_taxi

,passenger_name,tpep_pickup_datetime,tpep_dropoff_datetime,payment_method,passenger_count,trip_distance,fare_amount,tip_amount,tolls_amount
0,Pamela Duffy,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,Debit Card,6,3.34,13.0,2.76,0.00
1,Michelle Foster,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,Debit Card,1,1.80,16.0,4.00,0.00
2,Tina Combs,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,Debit Card,1,1.00,6.5,1.45,0.00
3,Anthony Ray,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,Cash,1,3.70,20.5,6.39,0.00
4,Brianna Johnson,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,Debit Card,1,4.37,16.5,0.00,0.00
...,...,...,...,...,...,...,...,...,...
22696,Austin Johnson,02/24/2017 5:37:23 PM,02/24/2017 5:40:39 PM,Cash,3,0.61,4.0,0.00,0.00
22697,Monique Williams,08/06/2017 4:43:59 PM,08/06/2017 5:24:47 PM,Cash,1,16.71,52.0,14.64,5.76
22698,Drew Graves,09/04/2017 2:54:14 PM,09/04/2017 2:58:22 PM,Debit Card,1,0.42,4.5,0.00,0.00
22699,Jonathan Copeland,07/15/2017 12:56:30 PM,07/15/2017 1:08:26 PM,Debit Card,1,2.36,10.5,1.70,0.00


데이터 확인

In [3]:
df_taxi.info() # 칼럼과 데이터타입의 구성 확인   # Dtype의 Object 타입에서 종종 결측치가 발견되는 경향 ↑  ('N/A', '-', '', 'Missing')를 포함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22701 entries, 0 to 22700
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   passenger_name         22701 non-null  object 
 1   tpep_pickup_datetime   22701 non-null  object 
 2   tpep_dropoff_datetime  22701 non-null  object 
 3   payment_method         22701 non-null  object 
 4   passenger_count        22701 non-null  int64  
 5   trip_distance          22701 non-null  float64
 6   fare_amount            22698 non-null  float64
 7   tip_amount             22701 non-null  float64
 8   tolls_amount           22701 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.6+ MB


In [4]:
 df_taxi.describe().astype(int)  # 데이터에 대한 기술통계량 확인 -> 보기 쉽게 int형으로 전환
 # fare_amount 컬럼과 passenger_count 컬럼의 값들에서 다소, 오류값이 있어보임.
 # passenger_count 에서의 max 값
 # fare_amount 에서의 min, max 값 정도

,passenger_count,trip_distance,fare_amount,tip_amount,tolls_amount
count,22701,22701,22698,22701,22701
mean,1,2,13,1,0
std,1,3,13,2,1
min,0,0,-120,0,0
25%,1,0,6,0,0
50%,1,1,9,1,0
75%,2,3,14,2,0
max,36,33,999,200,19


In [5]:
df_taxi.isnull().sum()

,0
passenger_name,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
payment_method,0
passenger_count,0
trip_distance,0
fare_amount,3
tip_amount,0
tolls_amount,0


fare_amount 칼럼에 대한 확인

In [6]:
print("fare amount 열의 빈도수 : ", df_taxi['fare_amount'].value_counts(ascending = False)) # 칼럼 내, 값들에 대한 빈도 확인

fare amount 열의 빈도수 :  fare_amount
6.0      1163
6.5      1089
5.5      1081
7.0      1067
7.5      1018
         ... 
73.5        1
94.2        1
66.5        1
450.0       1
16.8        1
Name: count, Length: 185, dtype: int64


In [7]:
df_taxi['fare_amount'].fillna(df_taxi['fare_amount'].mean(), inplace=True) # 결측값 처리
print("\n")
print("==="*6)
print("\n")
print(df_taxi['fare_amount'].isnull().sum()) # 결측값 처리 이후의, 칼럼에서의 null 값 관측
print("\n")
print("==="*6)
print("\n")
df_taxi #






0






/tmp/ipython-input-7-3502381269.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_taxi['fare_amount'].fillna(df_taxi['fare_amount'].mean(), inplace=True) # 결측값 처리


,passenger_name,tpep_pickup_datetime,tpep_dropoff_datetime,payment_method,passenger_count,trip_distance,fare_amount,tip_amount,tolls_amount
0,Pamela Duffy,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,Debit Card,6,3.34,13.0,2.76,0.00
1,Michelle Foster,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,Debit Card,1,1.80,16.0,4.00,0.00
2,Tina Combs,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,Debit Card,1,1.00,6.5,1.45,0.00
3,Anthony Ray,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,Cash,1,3.70,20.5,6.39,0.00
4,Brianna Johnson,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,Debit Card,1,4.37,16.5,0.00,0.00
...,...,...,...,...,...,...,...,...,...
22696,Austin Johnson,02/24/2017 5:37:23 PM,02/24/2017 5:40:39 PM,Cash,3,0.61,4.0,0.00,0.00
22697,Monique Williams,08/06/2017 4:43:59 PM,08/06/2017 5:24:47 PM,Cash,1,16.71,52.0,14.64,5.76
22698,Drew Graves,09/04/2017 2:54:14 PM,09/04/2017 2:58:22 PM,Debit Card,1,0.42,4.5,0.00,0.00
22699,Jonathan Copeland,07/15/2017 12:56:30 PM,07/15/2017 1:08:26 PM,Debit Card,1,2.36,10.5,1.70,0.00


In [8]:
Q1 = df_taxi['trip_distance'].quantile(0.25)
Q3 = df_taxi['trip_distance'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_distance_iqr = df_taxi[(df_taxi['trip_distance'] < lower_bound) | (df_taxi['trip_distance'] > upper_bound)]
print(f"IQR 기준 이상치 수: {len(outliers_distance_iqr)}")


IQR 기준 이상치 수: 2527
